In [6]:
import wandb
import pandas as pd

In [2]:
api = wandb.Api()

In [7]:
entity, project = "compositional-generalization-ut", "experiment-27"
runs = api.runs(entity + "/" + project)

In [8]:
# dump all runs to a csv file

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

runs_df.to_csv("project.csv")

In [20]:
for run in runs:
    history = run.scan_history()
    print(run.name)
    data = list(history)
    print(len(data))
    print(data[:5])
    break

tdec_1216embd_add_10000000_s1
973
[{'_step': 0, 'samples': {'artifact_path': 'wandb-client-artifact://gv141z9ywb0upn40rrsodr926em36k94245q544dgg0cpx15o148pjwj7izf7l6lj2mudipnq258pvhtl0oq889wfxfmt4rr8ktri8kqsolbwf3b6mqgmqk2japtcu0h/samples.table.json', 'ncols': 6, 'nrows': 50, 'path': 'media/table/samples_0_23e54421de223b29ede5.table.json', 'sha256': '23e54421de223b29ede5636acc2bc98fb547bbd4885e285722be3b47b725e141', 'size': 5035, '_latest_artifact_path': 'wandb-client-artifact://8fnfuy4iz0v66as7pdr9t64u1gryk3wamlirdwf5tzo27ot8ofuajrf801zt6is60i9u046iqcgcd7ux194dd9jeeircdvvvmm8b9oanz31pprzlt6iaer89yk6nxq6w:latest/samples.table.json', '_type': 'table-file'}, '_runtime': 7.4991936683654785, '_timestamp': 1728654860.8798006}, {'trainer/global_step': 19, '_step': 1, 'lr-AdamW/pg1': 2.8499999999999998e-05, 'lr-AdamW/pg2': 2.8499999999999998e-05, '_runtime': 15.326038837432861, '_timestamp': 1728654868.7066457}, {'trainer/global_step': 19, '_step': 2, 'lr-AdamW/pg1': 2.8499999999999998e-05, '